    2.3 Создадим матрицу корреспонденций районов.

In [1]:
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

In [2]:
# Воспользуемся итератором для чтения csv
iterator_matrix = pd.read_csv('matrix.csv', iterator=True, delimiter=';')

In [3]:
chunk_size = 15_000_000
chunks = []
while True:
    try:
        chunk = iterator_matrix.get_chunk(chunk_size)
        chunks.append(chunk)
    except StopIteration:
        print('stop iteration')
        break

stop iteration


In [4]:
# Соединим данные в один дата фрейм
df = pd.concat(chunks)
df.columns = ['time', 'depart', 'arrival', 'customers', 'customers_metro']
# df['time'] = pd.to_datetime(df['time'])

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 191345852 entries, 0 to 191345851
Data columns (total 5 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   time             object
 1   depart           int64 
 2   arrival          int64 
 3   customers        int64 
 4   customers_metro  int64 
dtypes: int64(4), object(1)
memory usage: 7.1+ GB


In [6]:
districts = pd.read_csv('zones.csv', delimiter=';')
districts.columns = ['zone', 'district']

districts.head()

,zone,district
0,914,277
1,915,277
2,615,277
3,539,167
4,208,167


In [7]:
zone_to_dist = df.merge(districts, left_on=['depart'], right_on=['zone'], how='left')

In [8]:
zone_to_dist.drop('depart', axis=1, inplace=True)
zone_to_dist.drop('zone', axis=1, inplace=True)
zone_to_dist.rename(columns = {'district': 'depart'}, inplace=True)


In [9]:
zone_to_dist = zone_to_dist.merge(districts,
                                  left_on=['arrival'], right_on=['zone'], how='left')

In [10]:
zone_to_dist.drop('arrival', axis=1, inplace=True)
zone_to_dist.drop('zone', axis=1, inplace=True)
zone_to_dist.rename(columns = {'district': 'arrival'}, inplace=True)

In [11]:
zone_to_dist = zone_to_dist.groupby(
                ['time', 'depart', 'arrival'], as_index=False).agg(
                {'customers': 'sum', 'customers_metro': 'sum'})
zone_to_dist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60473007 entries, 0 to 60473006
Data columns (total 5 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   time             object
 1   depart           int64 
 2   arrival          int64 
 3   customers        int64 
 4   customers_metro  int64 
dtypes: int64(4), object(1)
memory usage: 2.3+ GB


In [12]:
zone_to_dist.head()

,time,depart,arrival,customers,customers_metro
0,2014.08.01 00:00,1,1,269,0
1,2014.08.01 00:00,1,3,1,1
2,2014.08.01 00:00,1,44,21,0
3,2014.08.01 00:00,1,47,55,0
4,2014.08.01 00:00,1,67,1,0


In [14]:
# Запишем получившуюся матрицу в csv файл
zone_to_dist.to_csv('matrix_district.csv', encoding='utf-8', sep=';', index=False)